# Analysis/Cleaning

In [42]:
import duckdb as dd
import pandas as pd

In [53]:
con = dd.connect("project.db")

In [54]:
teams = con.sql("SELECT * FROM DIMTEAM").df()

In [55]:
teams["Conference"] = teams["Conference"].str.split().apply(lambda x: " ".join(x[:-1]))

In [56]:
colors = pd.read_csv("color_teams.csv")

In [57]:
teams["HexColor"] = colors["Hex_Code"]

In [58]:
con.sql("CREATE OR REPLACE TABLE DIMTEAM as SELECT * FROM teams")

In [30]:
rosters = con.sql("SELECT * FROM rosters").df()

## Roster Turnover

In [31]:
roster_groups = {name: list(group['PlayerID']) for name, group in rosters.sort_values(by=["Team", "Season", "Name"]).groupby(["Team", "Season"])}

In [32]:
roster_ids = {}
for key in roster_groups.keys():
    if key[0] not in roster_ids:
        roster_ids[key[0]] = {}
    roster_ids[key[0]][key[1]] = roster_groups[key]

In [33]:
def turnover_pct(prev, next):
    p = set(prev)
    n = set(next)

    num_same = len(n.intersection(p))

    return num_same/len(n)

In [34]:
roster_turnovers = {}
for key in roster_ids.keys():
    roster_turnovers[key] = {}

    sorted_keys = sorted(roster_ids[key].keys())
    for index, key2 in enumerate(sorted_keys):
        if index == 0:
            roster_turnovers[key][key2] = None
        else:
            roster_turnovers[key][key2] = turnover_pct(roster_ids[key][sorted_keys[index-1]], roster_ids[key][key2])

In [35]:
turnover = pd.DataFrame(roster_turnovers).reset_index().melt(id_vars=["index"], var_name="School", value_name="PctTurnover")

In [36]:
con.sql("""
    CREATE OR REPLACE TABLE ROSTERFACT as
    SELECT r.*, t.PctTurnover
    FROM ROSTERFACT r
    LEFT JOIN turnover t 
    ON r.Team=t.School AND r.Season=t.index
"""
       )

In [59]:
con.close()